Install the required packages by running the following command in the first cell of the notebook.

In [1]:
!pip install numpy pandas
!pip install pgvector
!pip install langchain langchain_google_vertexai transformers
!pip install google-cloud-aiplatform
!pip install psycopg2-binary
!pip install protobuf
!pip install shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 18.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 34.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 44.1 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10

To use the newly installed packages in this Jupyter runtime, it is recommended to restart the runtime. **Restart the kernel**

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

# Download and load the dataset

An AlloyDB cluster named cymbal-alloy-cluster is configured in this lab. To begin, let's locate the AlloyDB cluster's IP address.

On the Google Cloud console title bar, type "AlloyDB" in the Search field, then click AlloyDB in the Products & Pages section.

Locate the cluster named cymbal-alloy-cluster, and the primary instance named cymbal-master-instance. The private IP address of this instance serves as your access point for utilizing AlloyDB throughout the lab.

import necessary libraries

In [1]:
import os
import pandas as pd
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
import vertexai

PROJECT_ID = "qwiklabs-gcp-01-0205e348a520"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

Run the following code snippet to import the `psycopg2` library, which allows Python to interact with PostgreSQL databases, reads the CSV dataset into a pandas DataFrame, and finally saves the DataFrame to a table named products in the AlloyDB cluster.

In [2]:
import psycopg2

# Replace with your AlloyDB cluster credentials
cluster_ip_address = "10.12.0.2"
database_user = "postgres"
database_password = "postgres"

# Set environment variables for psql connection
os.environ["PGHOST"] = cluster_ip_address
os.environ["PGUSER"] = database_user
os.environ["PGPASSWORD"] = database_password

# Establish a connection to the database
try:
    conn = psycopg2.connect(
        host=cluster_ip_address,
        user=database_user,
        password=database_password
    )
    print("Connected to the database successfully!")
except Exception as e:
    print("Connection error:", e)
exit(1)

# Read the dataset from the URL
DATASET_URL = "https://github.com/GoogleCloudPlatform/python-docs-samples/raw/main/cloud-sql/postgres/pgvector/data/retail_toy_dataset.csv"
df = pd.read_csv(DATASET_URL)

# Select desired columns and drop missing values
df = df.loc[:, ["product_id", "product_name", "description", "list_price"]]
df = df.dropna()

# Save the DataFrame to the AlloyDB cluster
df.to_sql('products', con=f'postgresql://{cluster_ip_address}', if_exists='replace', index=False)

# Retrieve data from the 'products' table
cur = conn.cursor()
cur.execute("SELECT * FROM products")
results = cur.fetchall()

# Close the connection
conn.close()
print(results[5])

Connected to the database successfully!
('74a695e3675efc2aad11ed73c46db29b', 'Slip N Slide Triple Racer with Slide Boogies', 'Triple Racer Slip and Slide with Boogie Boards. The unit is 16 foot long. The unit has 3 sliding lanes.', 37.21)


# Generate Vector Embeddings using a Text Embedding Model

In this section, let's **preprocess** 
- product descriptions, 
- generate vector embeddings for them, 
- and store the embeddings along with other relevant data in a PostgreSQL database table for downstream analysis or applications.

Run the following code snippet to import the `RecursiveTextSplitter` class from the `LangChain library`, which is used for splitting text into smaller chunks. 

Iterate through each row in the DataFrame `df` and extract the **product ID** and **description** from each row.

Then, we will split each description into smaller chunks and will create a dictionary for each chunk.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Set up the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)

# Define the maximum number of documents to process
max_documents = 50  # Reduced limit to further control API usage
documents = []

# Create Document objects with product_id as metadata
for index, row in df.iterrows():
    product_id = row["product_id"]
    desc = row["description"]
    documents.append(Document(page_content=desc, metadata={"product_id": product_id}))

# Use the text splitter on a subset of documents (e.g., 40-50)
chunked = []
docs = text_splitter.split_documents(documents[40:max_documents])

# Collect split content along with product_id
for doc in docs:
    chunked.append({"product_id": doc.metadata["product_id"], "content": doc.page_content})

print(len(chunked))


34


Run the following code snippet to process product descriptions from a dataset by splitting them into smaller chunks, 
sending them to Vertex AI for embedding generation, and storing the retrieved embeddings back into the data structure.

In [13]:
from langchain_google_vertexai import VertexAIEmbeddings
from google.cloud import aiplatform
import time

embeddings_service = VertexAIEmbeddings(model_name="text-embedding-005")

# embeddings_service = VertexAIEmbeddings(
#     model_name="textembedding-gecko@latest",  # or @003 if required
#     project="qwiklabs-gcp-01-0205e348a520",                # optional if default is set
#     location="us-central1"                    # must be explicitly set
# )


# Helper function to retry failed API requests with exponential backoff.
def retry_with_backoff(func, *args, retry_delay=10, backoff_factor=2.5, **kwargs):  # Increased delay and backoff factor
    max_attempts = 10
    retries = 0
    for i in range(max_attempts):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"error: {e}")
            retries += 1
            wait = retry_delay * (backoff_factor**retries)
            print(f"Retry after waiting for {wait} seconds...")
            time.sleep(wait)

# Reduced batch size for API calls to manage quota limits
batch_size = 3
for i in range(0, len(chunked), batch_size):
    request = [x["content"] for x in chunked[i : i + batch_size]]
    response = retry_with_backoff(embeddings_service.embed_documents, request)
    # Store the retrieved vector embeddings for each chunk back.
    for x, e in zip(chunked[i : i + batch_size], response):
        x["embedding"] = e

# Store the generated embeddings in a pandas dataframe.
product_embeddings = pd.DataFrame(chunked)
product_embeddings.head()


,product_id,content,embedding
0,8a6d71be41e01b284294ec488508b414,All of our productsWalmartply with internation...,"[0.04086191579699516, -0.020511694252490997, -..."
1,8a6d71be41e01b284294ec488508b414,. Holds Up to 6 Decks Fun for the whole family...,"[0.01160380244255066, -0.021363208070397377, -..."
2,9648838f5badebb9fc0b07f89cc29394,Better circulate water through your pool with ...,"[-0.005303527694195509, 0.017071831971406937, ..."
3,9648838f5badebb9fc0b07f89cc29394,".25-inch fitting (11070), 2 strainer grids (11...","[-0.01586345210671425, 0.017979448661208153, -..."
4,9648838f5badebb9fc0b07f89cc29394,. Circulate water through your pool with the h...,"[0.0184821505099535, 0.01372526679188013, 0.00..."


Run the following command to enable AlloyDB integration.

In [14]:
!PROJECT_ID=$(gcloud config get-value project) && \
PROJECT_NUMBER=$(gcloud projects list --filter="name=$PROJECT_ID" --format="value(PROJECT_NUMBER)") && \
gcloud projects add-iam-policy-binding $PROJECT_ID \
--member="serviceAccount:service-$PROJECT_NUMBER@gcp-sa-alloydb.iam.gserviceaccount.com" \
--role="roles/aiplatform.user"

Updated IAM policy for project [qwiklabs-gcp-01-0205e348a520].
bindings:
- members:
  - serviceAccount:service-622450730797@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:service-622450730797@gcp-sa-aiplatform.iam.gserviceaccount.com
  - serviceAccount:service-622450730797@gcp-sa-alloydb.iam.gserviceaccount.com
  role: roles/aiplatform.user
- members:
  - serviceAccount:service-622450730797@gcp-sa-alloydb.iam.gserviceaccount.com
  role: roles/alloydb.serviceAgent
- members:
  - serviceAccount:qwiklabs-gcp-01-0205e348a520@qwiklabs-gcp-01-0205e348a520.iam.gserviceaccount.com
  role: roles/bigquery.admin
- members:
  - serviceAccount:622450730797@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-622450730797@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-622450730797@compute-system.iam.gserviceaccou

Back in the AlloyDB service page, click on the cluster named cymbal-alloy-cluster, then select AlloyDB Studio from the left-hand side menu, enter the following values to sign in, and click on AUTHENTICATE.

Field -> Value

Database -> postgres

User -> postgres

Password -> postgres

n the AlloyDB Studio, click on Editor tab at the top.

Enter the following command in the editor to grant the postgres user permission to execute the embedding function, install the google_ml_integration extension, and generate an embedding for the provided text using the textembedding-gecko model. Then, click on Run button at the top.

In [ ]:
GRANT EXECUTE ON FUNCTION embedding TO postgres;

CREATE EXTENSION IF NOT EXISTS google_ml_integration CASCADE;

SELECT embedding('text-embedding-005', 'AlloyDB is a managed, cloud-hosted SQL database service.');

Once the embeddings are successfully generated, click the Clear button at the top to clear the contents of the editor. Then, run the following query in the editor to prepare the database.

In [ ]:
CREATE EXTENSION IF NOT EXISTS vector;

DROP TABLE IF EXISTS product_embeddings;

click the Clear button at the top to clear the contents of the editor. Run the following query, to create the embeddings based on product descriptions.

In [ ]:
CREATE TABLE product_embeddings(
        product_id VARCHAR(1024) NOT NULL PRIMARY KEY,
        content TEXT,
        embedding vector(768)
    );


insert into product_embeddings(product_id, content, embedding)
SELECT
product_id,
description as content,
embedding('text-embedding-005', description) as embedding
from products
where product_id not in (select product_id from product_embeddings)
limit 10;

# Create Indexes for faster Similarity Search

Vector indexes can significantly speed up similarity search operations and avoid the brute-force exact nearest neighbor search that is used by default.

**Pgvector** comes with two types of indexes: **hnsw** and **ivfflat**.

In the Editor, run the following query to build the **HNSW index** on **product_embeddings** table using cosine similarity metric for faster search based on descriptions.

In [ ]:
-- Create an HNSW index on the `product_embeddings` table
CREATE INDEX ON product_embeddings
USING hnsw(embedding vector_cosine_ops)
WITH (m = 24, ef_construction = 100);


Next, run the query to create an **IVFFLAT** index on the **product_embeddings** table using cosine similarity for swift similarity searches among product descriptions.

In [ ]:
-- Create an IVFFLAT index on the `product_embeddings` table
CREATE INDEX ON product_embeddings
USING ivfflat(embedding vector_cosine_ops)
WITH (lists = 100);

Now, we will conduct the similarity search. The provided code identifies products most relevant to the user's query based on their textual descriptions. To ensure relevant results.

In [ ]:
with e as (
SELECT
    *
FROM
    product_embeddings
ORDER BY
    embedding <-> CAST(embedding('text-embedding-005','Playing card games') AS vector(768)) asc
LIMIT
    5
)
select
*
from products
where product_id in (select e.product_id from e);

Finally, the top matches are displayed as a list, making it easy to find products that fit your search and budget. You'll see the product name, price, and a brief description for each result, giving you a quick overview of your options.

# LLMs and LangChain

### Use case 1: Building an AI-curated contextual hybrid search


Combine natural language query text with regular relational filters to create a powerful hybrid search.

Build a user query with english text and the price filters.

In [37]:
# Please fill in these values.
user_query = "Do you have a toy set that teaches numbers and letters to kids?"  # @param {type:"string"}
min_price = 10  # @param {type:"integer"}
max_price = 100  # @param {type:"integer"}

Generate the vector embedding for the user query.

In [38]:
qe = embeddings_service.embed_query(user_query)

Use pgvector to find similar products. The pgvector similarity search operators provide powerful semantics to combine the vector search operation with regular query filters in a single SQL query.

In [39]:
import psycopg2
from psycopg2 import sql
from pgvector.psycopg2 import register_vector
import pandas as pd

def main(user_query, min_price, max_price):
    try:
        # AlloyDB cluster connection details (replace with your actual values)
        cluster_ip_address = "10.12.0.2"
        database_user = "postgres"
        database_password = "postgres"

        # Connect to AlloyDB cluster
        conn = psycopg2.connect(
            host=cluster_ip_address,
            user=database_user,
            password=database_password
        )

        # Register the vector type
        register_vector(conn)

        # Get the query embedding
        qe = embeddings_service.embed_query(user_query)

        # Check if qe is valid
        if not qe:
            print("Error: The query embedding is empty.")
            return

        # Perform the similarity search and filtering
        cur = conn.cursor()
        similarity_threshold = 0.5  # Increased threshold for broader matching
        num_matches = 50

        # Modify the SQL query for indexed similarity search
        cur.execute(
            """
            WITH vector_matches AS (
                SELECT product_id, embedding <=> %s::vector AS distance
                FROM product_embeddings
                WHERE embedding <=> %s::vector < %s
                ORDER BY distance ASC
                LIMIT %s
            )
            SELECT product_name, list_price, description
            FROM products
            WHERE product_id IN (SELECT product_id FROM vector_matches)
            AND list_price >= %s AND list_price <= %s
            """,
            (qe, qe, similarity_threshold, num_matches, min_price, max_price)
        )
        results = cur.fetchall()

        # Check if any results are retrieved
        if not results:
            print("No results found. Try adjusting the similarity threshold or checking the data.")
            return

        # Process the results
        matches = []
        for r in results:
            try:
                list_price = round(float(r[1]), 2)  # Attempt conversion and rounding
            except ValueError:
                list_price = r[1]  # Use original value if conversion fails
            matches.append({
                "product_name": r[0],
                "list_price": list_price,
                "description": r[2]
            })

        # Display the results
        matches_df = pd.DataFrame(matches)
        print(matches_df.head(5))

    except Exception as e:
        print(f"Error during database operations: {e}")
    finally:
        # Close the connection
        if conn:
            conn.close()

# Call the main function
main("Do you have a toy set that teaches numbers and letters to kids?", 25, 100)

No results found. Try adjusting the similarity threshold or checking the data.


In [ ]:
The Result Is Different than the Lab Result , Could be cause by using a different Text Embedding Model, Not Sure

Use LangChain to summarize and generate a high-quality prompt to answer the user query.

After finding the similar products and their descriptions using pgvector, the next step is to use them for generating a prompt input for the LLM model. Since individual product descriptions can be very long, they may not fit within the specified input payload limit for an LLM model. The MapReduceChain from the LangChain framework is used to generate and combine short summaries of similarly matched products. The combined summaries are then used to build a high-quality prompt for an input to the LLM model.

In [35]:
from IPython.display import display, Markdown
from langchain_core.documents.base import Document
from langchain.chains.summarize import load_summarize_chain
from langchain_google_vertexai import VertexAI
from langchain_core.prompts import PromptTemplate

# Mock matches data
matches = [
    {"product_name": "Alphabet Learning Toy", "price": 30, "features": "Teaches letters and numbers."},
    {"product_name": "Number Puzzle", "price": 20, "features": "Interactive puzzle for number learning."},
]

# LangChain setup
llm = VertexAI(model_name="gemini-2.0-flash-001")

map_prompt_template = """
            You will be given a detailed description of a toy product.
            This description is enclosed in triple backticks (```).
            Using this description only, extract the name of the toy,
            the price of the toy and its features.

            ```{text}```
            SUMMARY:
            """
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                You will be given a detailed description of different toy products
                enclosed in triple backticks (```) and a question enclosed in
                double backticks(``).
                Select one toy that is most relevant to answer the question.
                Using that selected toy description, answer the following
                question in as much detail as possible.
                You should only use the information in the description.
                Your answer should include the name of the toy, the price of the toy
                and its features. Your answer should be less than 200 words.
                Your answer should be in Markdown in a numbered list format.

                Description:
                ```{text}```

                Question:
                ``{user_query}``

                Answer:
                """
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text", "user_query"]
)

# Convert matches to LangChain documents
docs = [
    Document(page_content=f"Name: {match['product_name']}, Price: {match['price']}, Features: {match['features']}")
    for match in matches
]

# Load and invoke the chain
chain = load_summarize_chain(
    llm, chain_type="map_reduce", map_prompt=map_prompt, combine_prompt=combine_prompt
)

# User query
user_query = "Do you have a toy set that teaches numbers and letters to kids?"

# Invoke the chain
output = chain.invoke({
    "input_documents": docs,
    "user_query": user_query,
})

# Extract and display the output
answer = output.get('output_text', ' ')
display(Markdown(answer))


Here's the answer based on the provided toy descriptions:

1.  The toy set that teaches numbers and letters to kids is the **Alphabet Learning Toy**.
2.  The price of the Alphabet Learning Toy is 30.
3.  The Alphabet Learning Toy teaches letters and numbers.


### Use case 2: Adding AI-powered creative content generation

Use knowledge from the existing dataset to generate new AI-powered content from an initial prompt.

A third-party seller on the retail platform wants to use the AI-powered content generation to create a detailed description of their new bicycle product.

In [30]:
# Please fill in these values.
creative_prompt = "A bicycle with brand name 'Roadstar bike' for kids that comes with training wheels and helmet." 

Leverage the pgvector similarity search operator to find an existing product description that closely matches the new product specified in the initial prompt.

In [32]:
import psycopg2
from pgvector.psycopg2 import register_vector

def main():
    try:
        # AlloyDB cluster connection details
        cluster_ip_address = "10.12.0.2"
        database_user = "postgres"
        database_password = "postgres"

        # Connect to AlloyDB cluster
        conn = psycopg2.connect(
            host=cluster_ip_address,
            user=database_user,
            password=database_password
        )

        # Register the vector type
        register_vector(conn)

        # Get the query embedding
        qe = embeddings_service.embed_query(creative_prompt)

        # Check if qe is a valid embedding
        if not qe:
            print("Error: The query embedding is empty.")
            return

        # Set similarity threshold
        similarity_threshold = 0.5
        matches = []

        # Perform the similarity search and filtering
        cur = conn.cursor()
        cur.execute(
            """
            WITH vector_matches AS (
                SELECT product_id, embedding <=> %s::vector AS distance
                FROM product_embeddings
                WHERE embedding <=> %s::vector < %s
                ORDER BY distance ASC
                LIMIT 1
            )
            SELECT description FROM products
            WHERE product_id IN (SELECT product_id FROM vector_matches)
            """,
            (qe, qe, similarity_threshold)
        )

        results = cur.fetchall()

        # Process the results
        for r in results:
            matches.append(r[0])

        if not matches:
            print("No matches found.")
        else:
            print("Matches found:", matches)

    except Exception as e:
        print(f"Error during database operations: {e}")
    finally:
        # Close the connection if it was established
        if conn:
            conn.close()

# Call the main function
main()


Matches found: ['Turn any small bicycle into an instrument for learning to ride with the Schwinn 12"-20" Training Wheels. They feature a slotted design to fit 12" to 20" bikes. The training wheels are easy to assemble, install and remove, so that when your little one is able to ride without assistance, you can take them off. These bicycle training wheels include steel brackets and rubber tires that can stand up to heavy use. Training Wheels, Fits 12 inches - 20 inches bicycles. Est. 1895. Durable Construction: Steel brackets stand up to heavy use. Customizable: Two sets of wheel decals included. Features: Fits Most Childrens Bicycles: Intended for 12 inch - 20 inch bicycles. Steel Brackets: Offer increased durability. Includes two sets of wheel decals: Learn how to ride in style - see images below. Easy to Adjust: Slotted design for size adjustment. Includes: One pair of training wheels, four decals, installation instructions, and all mounting hardware. Tools required: Adjustable wrenc

Use the existing matched product description as the prompt context to generate new creative output from the LLM.

In [34]:
from IPython.display import display, Markdown
from langchain_google_vertexai import VertexAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

# Define the template
template = """
            You are given descriptions about some similar kind of toys in the context.
            This context is enclosed in triple backticks (```).
            Combine these descriptions and adapt them to match the specifications in
            the initial prompt. All the information from the initial prompt must
            be included. You are allowed to be as creative as possible,
            and describe the new toy in as much detail. Your answer should be
            in markdown in lists and less than 200 words.

            Context:
            ```{context}```

            Initial Prompt:
            {creative_prompt}

            Answer:
        """

prompt = PromptTemplate(
    template=template, input_variables=["context", "creative_prompt"]
)

# Define the LLM
llm = VertexAI(model_name="gemini-2.0-flash-001", temperature=0.7)

# Example `matches` list
matches = [
    {"description": "This is a toy description 1."},
    {"description": "This is a toy description 2."},
    {},  # Missing `description`
    "Invalid item"  # Not a dictionary
]

# Construct the context by extracting valid descriptions
context = "\n".join(
    item["description"] for item in matches if isinstance(item, dict) and "description" in item
)

# Define the creative prompt
creative_prompt = "Describe a toy that is suitable for both indoor and outdoor play."

# Use RunnableSequence for chaining
llm_chain = RunnableSequence(prompt | llm)

# Invoke the chain
answer = llm_chain.invoke({
    "context": context,
    "creative_prompt": creative_prompt,
})

# Display the answer in Markdown format
display(Markdown(answer))



Here's a toy description suitable for both indoor and outdoor play, combining elements from toy descriptions 1 & 2:

*   **The "AdventureSphere":** A large, inflatable ball made of durable, weather-resistant material.

*   **Versatile Play:** Perfect for rolling, bouncing, and chasing in the backyard, park, or even inside on a rainy day.

*   **Imaginative Designs:** Available in various vibrant colors and patterns, including glow-in-the-dark options for evening fun. Some models feature printed maps or constellations to spark curiosity.

*   **Safe and Durable:** Constructed with reinforced seams and a secure valve to prevent leaks. Phthalate-free and non-toxic.

*   **Interactive Features:** Certain AdventureSpheres include built-in sound effects or light-up elements activated by motion, adding an extra layer of engagement.
